In [1]:
import numpy as np
import pandas

ranking_path = "/home/sasce/PycharmProjects/GitHubClassificationDataset/data/pairrank/mean_std_ranking.csv"
ranking = pandas.read_csv(ranking_path)
ranking = ranking.sort_values('mean', ascending=False)
ranking['description'] = ranking['description'].fillna('')
cluster_feature = 'mean'

embedding_key = 'q_id'

In [2]:
from src.ml.embeddings import FastTextEmbedding, W2VEmbedding, BERTEmbedding

#embedding = FastTextEmbedding('/home/sasce/PycharmProjects/ComponentSemantics/data/models/fastText/wiki.en.bin')
#embedding = W2VEmbedding('/home/sasce/PycharmProjects/GitHubClassificationDataset/data/models/github_pbg_w2v.vec', size=200)

embedding = W2VEmbedding('/home/sasce/PycharmProjects/GitHubClassificationDataset/data/models/github_pbg_w2v.vec',size=200,
                         preprocessing='upper')


In [14]:
X = [list(embedding.get_embedding(x)) + [m] for x,m in zip(ranking[embedding_key], ranking['mean'])]


In [19]:
from scipy.cluster.hierarchy import dendrogram, linkage, to_tree
from matplotlib import pyplot as plt

linked = linkage(X, 'single')

In [24]:
tree = to_tree(linked, rd=True)

In [29]:
pre_order = tree.pre_order()

In [35]:
res = []
def get_edges(node):
    left = node.left
    right = node.right
    res = []
    if left:
        a = [(left.id, node.id)]
        b = get_edges(left)
        res.extend(a+b)
    if right:
        c = [(right.id, node.id)]
        d =  get_edges(right)
        res.extend(c+d)

    return res

res = list(get_edges(tree))

In [ ]:
plt.figure(figsize=(10, 7))
dendrogram(linked,
            orientation='top',
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()

In [16]:

import numpy
import plotly.figure_factory as ff
fig = ff.create_dendrogram(numpy.array(X), labels=ranking['topic'].to_list())
fig.update_layout(width=800, height=500)
fig.show()